## Continous time Quantum Monte Carlo of a single spin

The Hamiltonian of a single spin in a magnetic field is
$$
H = -\frac{h}{2} \sigma_z - \frac{\Gamma}{2} \sigma_x.
$$

### Monte Carlo Implementation
The analytical solution for the magnetization $\langle \sigma_x \rangle= \frac{2}{\beta}\partial_\Gamma \log Z$ is
$$
\langle \sigma_x \rangle = \frac{\Gamma}{\sqrt{\Gamma^2+h^2}} \tanh\left( \frac{\beta}{2} \sqrt{h^2+\Gamma^2} \right).
$$
We map the quantum system to a classical continous 1D spin chain of length $\beta$. The total energy of the system is
$$
E_\mathrm{tot} = -\frac{h}{2} S_z^\mathrm{tot}- \frac{2n}{\beta}\log \left(\frac{d\tau \Gamma}{2}\right),
$$
where $2n$ is the number of domain walls and $\sigma(\tau)$ is the magnetization, and $S_z^\mathrm{tot}=\frac{1}{\beta}\int_0^\beta d\tau \sigma(\tau)$ the total magnetization. From the above result we can calculate the acceptance probabilities
$$
A(X \rightarrow Y) = \min\left( 1, \frac{V_Y T(Y\rightarrow X)}{V_X T(X\rightarrow Y)} \right).
$$
For adding two domain walls we get
$$
\frac{V_Y}{V_X} = \exp({-\beta \Delta E}) = \left(\frac{d\tau\Gamma}{2}\right)^2 \exp({\frac{\beta h}{2} \mathop{\Delta S_z^\mathrm{tot}}}),
$$
with the transition probability from $Y(2n+2) \rightarrow X(2n)$ given by the probability of removing two domain walls, selected subsequently out of the $2n+2$ available transitions
$$
\ T(Y\rightarrow X)=\frac{1}{\left(2n+1\right) \left(2n+2\right)},
$$
and $T(X\rightarrow Y)$ given by the probability of picking two imaginary time locations $\tau_1, \ \tau_2$,
$$
T(X\rightarrow Y)=\left(\frac{\mathop{d\tau}}{\beta}\right)^2.
$$
Therefore, the acceptance probability for adding two domain walls is
$$
A(X(2n) \rightarrow Y(2n+2)) = \min\left( 1, \frac{\exp(\beta h \mathop{\Delta S_z^\mathrm{tot}/2}) \left(\frac{\Gamma \beta}{2}\right)^2}{(2n+1)(2n+2)} \right) ,
$$
and for removing two domain walls it is
$$
A(X(2n) \rightarrow Y(2n-2)) = \min\left( 1, \frac{\exp(\beta h \mathop{\Delta S_z^\mathrm{tot}/2})2n(2n-1)}{ \left(\frac{\Gamma \beta}{2}\right)^2} \right).
$$
The average magnetization depends on the number of domain walls $2n$:
$$
\langle \sigma^x \rangle = \frac{2 \langle 2n \rangle}{\beta \Gamma}.
$$
This formula can be derived starting from $\langle \sigma^x\rangle = \frac{2}{\beta}\partial_\Gamma \log Z$ and writing the parition function in terms of configurations, $Z = \sum_C e^{-\beta E[C]}$, with $E[C]=-\frac{h}{2} S_z^\mathrm{tot}- \frac{2n}{\beta}\log \left(\frac{d\tau \Gamma}{2}\right)$ as above.

In [28]:
import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt
import copy
from sys import stdout
%matplotlib inline
plt.rcParams['figure.figsize'] = 16, 9


# Only needed for Mac OS users
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)

### Analytical solution

In [29]:
sx = lambda gamma,h,beta: gamma/np.sqrt(gamma**2+h**2)*np.tanh(beta*np.sqrt(h**2+gamma**2)/2)

### Worldline class for simulation

In [30]:
class Worldline:

    def __init__(self,beta,gamma,h=0):
        self.beta  = beta     # inverse temperature \beta
        self.gamma = gamma    # transverse field \Gamma
        self.h     = h        # magnetic field h
        self.s0    = 1        # spin at \tau=0 (whether the first spin is up or down)
        self.kinks = []       # position of kinks (domain walls), sorted from 0 to beta
        self.magnobs = []     # observations (measurements) of the magnetization

    def sz(self, kinks, s0):
        # Compute the magnetization of a given configuration (i.e. given first spin and domain walls)
        # the magnetization is given by S^z = 1/\beta \int_0^\beta d\tau \sigma(\tau)
        if len(kinks) > 0: #if we have a kink we have a sign change and between the kinks we have a constant sign
            sz = s0 *(kinks[0]  + (self.beta -kinks[-1])) # start---|.....|---end before and after the frist resp last kink we have the starting spin and this group we can already add to our magnetization. so ist the starting spin and self.beta ist the amount of soins in our system
            for i in range(len(kinks)-1): # we go from 1 till n-1 kins since we used the frist and last before. betwee nkins we have the same spin so we can just take the psitional difference betwee nthe kins and multiply it with s0
                sz += s0 * (-1)**(i+1) * (kinks[i+1] - kinks[i]) # between the kinks we have a constant sign and since s0 ist the sarting point we need to change the spin after everx kind with (-1)**(i+1)
                sz /= self.beta # we have to divide by beta since we have a constant spin between the kinks and we need to normalize it
        else: # if we have no kinks we have a constant spin
            sz = s0 * self.beta
        return sz 

    def szdifference_add(self, ta, tb):
        """
        Compute the difference in magnetization when adding two domain walls at imaginary times ta and tb.
        Because of periodic boundary conditions, they have to be added in pairs.

        Params
        ------
        ta : float, 0 <= ta <= self.beta
            Imaginary time of first new kink.
        tb : float, 0 <= tb <= self.beta
            Imaginary time of second new kink.

        """
        # add kinks at times ta and tb and take care of the initial spin s0
        if ta > tb:
            new_s0 = -1*self.s0 #the first spin is between the DW and so the sign has to be flipped
        else:
            new_s0 = self.s0 #the fist spin has the same sign with the addition of the new kinks
        new_kinks = copy.copy(self.kinks)
        new_kinks += (ta, tb)
        new_kinks.sort()
        # compute difference in total magnetization
        return self.sz(new_kinks, new_s0) - self.sz(self.kinks, self.s0)

    def szdifference_rem(self, i, j):
        """
        Compute the difference in magnetization when removing the two domain walls i and j.
        Because of periodic boundary conditions, they have to be removed in pairs.

        Params
        ------
        i : int, 0 <= i < len(self.kinks)
            Imaginary time of first kink to remove.
        j : int, 0 <= j < self.kinks
            Imaginary time of second kink to remove.

        """
        # remove two domain walls and take care of initial spin s0
        new_kinks = copy.copy(self.kinks)
        if i > j: # we have to delete first the higher index. If not we change the indexing of the second kins to i-1
            del new_kinks[i]
            del new_kinks[j]
            new_s0 = -1*self.s0 #the first spin is between the DW and so the sign has to be flipped
        else:
            del new_kinks[j]
            del new_kinks[i]
            new_s0 = self.s0
        # Compute difference in Sz
        return self.sz(new_kinks, new_s0) - self.sz(self.kinks, self.s0)

    def insertUpdate(self):                                             ######## where are the 2 in the calcolations? #########
        """
        Do an update that inserts two domain walls if accepted.

        Procedure:
        1. propose two new domain walls in [0, self.beta].
        2. compute acceptance probability A(X(n) -> Y(n+2))
        3. if proposition is accepted, add the new kinks and take care of initial spin

        """
        # 1. propose to add kinks at times ta, tb
        ta = rnd.uniform(0,self.beta)
        tb = rnd.uniform(0,self.beta)
        n = len(self.kinks) # number of kinks in the system

        # 2. compute acceptance probability A(X(n) -> Y(n+2))
        if self.h == 0:
            ratio = 1 # if h=0, the exponential is e^0 and we dont need to compute it
        else:
            ratio = np.exp(self.beta * 0.5 * self.h * self.szdifference_add(ta, tb)) # we can make the approxiamtion of H beta which than can be insertet in the ratio Vx/Vy, where H = deltaE = 1/2*h*deltaSz
        p_acc = ratio * (self.beta * self.gamma/2)**2 / ((n +1) * (n +2)) # the ratio between the energy changes and the removing and adding prob gives the next step acceptance probability

        # 2. if accepted, add the two domain walls and take care of initial spin
        if rnd.rand() < p_acc:
            self.kinks += (ta, tb)
            self.kinks.sort()
            if ta > tb:
                self.s0 *= -1

    def removeUpdate(self):
        """
        Do an update that removes two domain walls if accepted.

        Procedure:
        1. propose to remove the two domain walls at positions i, j
        2. compute acceptance probability A(X(n) -> Y(n-2))
        3. if proposition is accepted, remove the kinks and take care of the initial spin

        """
        # if no kinks, do nothing
        if len(self.kinks) == 0:
            return

        # 1. propose to remove kinks at position i, j in the list of kinks
        i = rnd.randint(0, len(self.kinks))
        j = rnd.randint(0, len(self.kinks))
        while i == j:
            j = rnd.randint(0, len(self.kinks))
        n = len(self.kinks) # number of kinks in the system

        # 2. compute acceptance ratio
        if self.h == 0:
            ratio = 1
        else:
            ratio = np.exp(self.beta * 0.5 * self.h * self.szdifference_rem(i, j))
        p_acc = ratio * n * (n-1) / (self.gamma * self.beta / 2)**2 # the ratio between the energy changes and the removing and adding prob gives the next step acceptance probability

        # 3. if accepted, remove the two kinks and take care of initial spin
        if rnd.rand() < p_acc:
            if i > j:
                del self.kinks[i]
                del self.kinks[j]
                self.s0 *= -1
            else:
                del self.kinks[j]
                del self.kinks[i]

    def update(self):
        # perform a MC update, with equal probability one that removes or adds
        # domain walls (if accepted)
        if rnd.rand() < 0.5:
            self.insertUpdate()
        else:
            self.removeUpdate()

    def measure(self):
        # measure the magnetization of the current configuration
        self.magnobs.append(2 * len(self.kinks) / (self.beta*self.gamma)) # Measure the average of sigma_x

### Binning analysis

In [31]:
def binning_analysis(samples, n_levels):
    """
    Perform a binning analysis over samples and return an array of the error estimate at each binning level.

    """
    bins = np.array(samples)
    errors = np.zeros(n_levels + 1)
    errors[0] = np.std(bins) / np.sqrt(len(bins) - 1)
    for k in range(n_levels):
        bins = np.array([(bins[2*i]+bins[2*i+1])/2. for i in range(len(bins)//2)])
        errors[k+1] = np.std(bins) / np.sqrt(len(bins) - 1)
    # calculate autocorrelation time
    tau = 0.5*(errors[-1]**2/np.std(samples)**2*(len(samples)-1.)-1.)
    return errors, tau

### Main part

In [32]:
steps = int(2**17)           # number of Monte Carlo steps
n_levels = 9                 # number of binning levels
thermsteps = steps//5        # number of thermalization steps
beta = 1.                    # inverse temperature \beta
h = 1.                       # magnetic field h
results = []

for gamma in np.linspace(0.05, 6., 100):
    wl = Worldline(beta, gamma, h)
    for i in range(thermsteps):
        # thermalize
        wl.update()
    for i in range(steps):
        # update the worldline and measure the magnetization
        wl.update()
        wl.measure()
    mean = np.mean(wl.magnobs)
    errors, tau = binning_analysis(wl.magnobs, n_levels)
    print('Gamma =',"{:06.5f}".format(gamma), ', <sigma_x> =',"{:06.5f}".format(mean), '+-', '{:06.5f}'.format(errors[-1]), ", tau = ",'{:06.5f}'.format(tau))
    results.append([gamma, mean, errors[-1], tau])

results = np.transpose(np.array(results))

Gamma = 0.05000 , <sigma_x> = 0.02197 +- 0.00602 , tau =  0.85200
Gamma = 0.11010 , <sigma_x> = 0.04379 +- 0.00584 , tau =  0.90744
Gamma = 0.17020 , <sigma_x> = 0.08212 +- 0.00728 , tau =  1.30437
Gamma = 0.23030 , <sigma_x> = 0.10164 +- 0.00711 , tau =  1.38108
Gamma = 0.29040 , <sigma_x> = 0.13546 +- 0.00726 , tau =  1.36994
Gamma = 0.35051 , <sigma_x> = 0.14723 +- 0.00624 , tau =  1.03883
Gamma = 0.41061 , <sigma_x> = 0.18336 +- 0.00688 , tau =  1.26580
Gamma = 0.47071 , <sigma_x> = 0.21246 +- 0.00713 , tau =  1.37624
Gamma = 0.53081 , <sigma_x> = 0.23739 +- 0.00729 , tau =  1.49177
Gamma = 0.59091 , <sigma_x> = 0.25223 +- 0.00679 , tau =  1.31154
Gamma = 0.65101 , <sigma_x> = 0.29509 +- 0.00658 , tau =  1.11625
Gamma = 0.71111 , <sigma_x> = 0.31809 +- 0.00681 , tau =  1.26668
Gamma = 0.77121 , <sigma_x> = 0.35483 +- 0.00726 , tau =  1.47464
Gamma = 0.83131 , <sigma_x> = 0.37481 +- 0.00735 , tau =  1.55610
Gamma = 0.89141 , <sigma_x> = 0.38549 +- 0.00672 , tau =  1.30338
Gamma = 0.

In [33]:
# Visualize

plt.figure(figsize=(10, 7))
plt.errorbar(results[0],results[1],results[2],fmt='.',label='Monte Carlo', capsize=4)
plt.plot(results[0],sx(results[0],h,beta),label='exact')
plt.xlabel('$\\Gamma$')
plt.ylabel('$\\langle \\sigma_x \\rangle$')
plt.title('Ising spin in transverse field ($\\beta='+str(beta)+'$)')
plt.legend(loc='best')
plt.show()

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 1000x700 with 1 Axes>